In [1]:
library(robets)
library(dplyr)
library(prophet)
library(fmtr)
library(forecast)
outlier_multi <- function(x) {
    if (x > 0) {
        return(1 + x)
    } else {
        return(exp(x))
    }
}
model <- function(type, out = FALSE) {
    ts <- 40
    m <- 4
    sigma <- 0.05
    alpha <- 0.36
    beta <- 0.21
    phi <- 0.9
    gamma <- 0.2
    l_0 <- 1
    b_0 <- 0.05
    s_ad <- c(-0.01, 0.01, 0.03, -0.03)
    s_m <- c(0.99, 1.01, 1.03, 0.97)
    h <- 1
    if (out) {
        epsilon <- 0.05 #Fraction of outliers
        K <- 20
        u_out <- unlist(lapply(rep(0, ts + 1), function(x)
                if (runif(1, 0, 1) < (1 - epsilon)) 0
                else rnorm(1, 0, K^2 * sigma^2)))
    } else {
        u_out <- rep(0, ts + 1)
    }
    e <- rnorm(ts + 1, 0, sigma^2)
    b <- rep(0, ts + 1)
    b[1] <- b_0
    l <- rep(0, ts + 1)
    l[1] <- l_0
    s <- rep(0, ts + m + 1)
    h_m <- floor((h - 1) %% m) + 1
    y <- rep(NA, ts + 1)
    u <- rep(0, ts + 1)
    y_pred <- rep(NA, ts + 1)
    first_char <- unlist(strsplit((type), split = ""))[1]

    if (type %in% c("ADN", "MDN", "ANN", "MNN", "AAN", "MAN")) {
        if (type %in% c("ANN", "MNN")) {
            #None Trend
            phi <- 0
            if (first_char  == "A") {
                for (t in seq(2, ts + 1)) {
                    u[[t]] <- l[[t - 1]]
                    l[[t]] <- l[[t - 1]] + alpha * e[[t]]
                }
                y <- u + e + u_out
            } else {
                for (t in seq(2, ts + 1)) {
                    u[[t]] <- l[[t - 1]]
                    l[[t]] <- l[[t - 1]] * (1 + alpha * e[[t]] * u[[t]])
                }
                y <- u * unlist(lapply((e + u_out), outlier_multi))
            }
        } else if (type %in% c("AAN", "MAN")) {
            # Additive Trend
            phi <- 1
            if (first_char  == "A") {
                for (t in seq(2, ts + 1)) {
                    u[[t]] <- l[[t - 1]] + b[[t - 1]]
                    l[[t]] <- l[[t - 1]] + b[[t - 1]] +  alpha * e[[t]]
                    b[[t]] <- b[[t - 1]] + alpha * beta * e[[t]]
                }
                y <- u + e + u_out
            } else {
                for (t in seq(2, ts + 1)) {
                    u[[t]] <- l[[t - 1]] + b[[t - 1]]
                    l[[t]] <- l[[t - 1]] + b[[t - 1]] +  alpha * e[[t]] * u[[t]]
                    b[[t]] <- b[[t - 1]] + alpha * beta * e[[t]] * u[[t]]
                }
                y <- u * unlist(lapply((e + u_out), outlier_multi))
            }
        } else {
            # Damped Trend
            if (first_char  == "A") {
                for (t in seq(2, ts + 1)) {
                    u[[t]] <- l[[t - 1]] + b[[t - 1]]
                    b[[t]] <- phi * b[[t - 1]] + alpha * beta * e[[t]]
                    l[[t]] <- l[[t - 1]] * b[[t - 1]] +  alpha * e[[t]]
                    }
                y <- u + e + u_out
            } else {
                for (t in seq(2, ts + 1)) {
                    u[[t]] <- l[[t - 1]] + b[[t - 1]]
                    b[[t]] <- phi * b[[t - 1]] + alpha * beta * e[[t]] * u[[t]]
                    l[[t]] <- l[[t - 1]] * b[[t - 1]] +  alpha * e[[t]] * u[[t]]
                }
                y <- u * unlist(lapply((e + u_out), outlier_multi))
            }
        }
        for (t in seq(2, ts + 1)) {
            l[[t]] <- alpha * y[[t]] +
             (1 - alpha) * (l[[t - 1]] + phi * b[[t - 1]])
            b[[t]] <- beta * (l[[t]] - l[[t - 1]]) +
             (1 - beta) * phi * b[[t - 1]]
            y_pred[t + h] <- l[[t]] + phi * b[[t]]
        }
    } else if (type %in% c("ANA", "MNA", "AAA", "MAA", "ADA", "MDA")) {
        s[1:4] <- s_ad
        #Underlying model
        if (type %in% c("ANA", "MNA")) {
            #None Trend
            phi <- 0
            if (first_char  == "A") { 
                for (t in seq(2, ts + 1)) {
                    t_offset <- t + m - 1
                    s[[t_offset]] <- s[[t_offset - m]] + gamma * e[[t]]
                    u[[t]] <- l[[t - 1]] + s[[t_offset - m]]
                    l[[t]] <- l[[t - 1]] + alpha * e[[t]]
                }
                y <- u + e + u_out
            } else {
                for (t in seq(2, ts + 1)) {
                        t_offset <- t + m - 1
                        u[[t]] <- l[[t - 1]] + s[[t_offset - m]]
                        s[[t_offset]] <- s[[t_offset - m]] +
                         gamma * e[[t]] * u[[t]]
                        l[[t]] <- l[[t - 1]] + alpha * e[[t]] * u[[t]]
                }
                y <- u * unlist(lapply((e + u_out), outlier_multi))
            }
        } else if (type %in% c("AAA", "MAA")) {
            # Additive Trend
            phi <- 1
            if (first_char  == "A") {
                for (t in seq(2, ts + 1)) {
                    t_offset <- t + m - 1
                    u[[t]] <- l[[t - 1]] + b[[t - 1]] + s[[t_offset - m]]
                    s[[t_offset]] <- s[[t_offset - m]] + gamma * e[[t]]
                    b[[t]] <- b[[t - 1]] + alpha * beta * e[[t]]
                    l[[t]] <- l[[t - 1]] + b[[t - 1]] + alpha * e[[t]]
                }
                y <- u + e + u_out
            } else {
                for (t in seq(2, ts + 1)) {
                    t_offset <- t + m - 1
                    u[[t]] <- l[[t - 1]] + b[[t - 1]] + s[[t_offset - m]]
                    s[[t_offset]] <- s[[t_offset - m]] + gamma * e[[t]] * u[[t]]
                    b[[t]] <- b[[t - 1]] + alpha * beta * e[[t]] * u[[t]]
                    l[[t]] <- l[[t - 1]] + b[[t - 1]] + alpha * e[[t]] * u[[t]]
                }
                y <- u * unlist(lapply((e + u_out), outlier_multi))
            }
        } else {
            #Damped Trend
            if (first_char  == "A") {
                for (t in seq(2, ts + 1)) {
                    t_offset <- t + m - 1
                    u[[t]] <- l[[t - 1]] + b[[t - 1]] + s[[t_offset - m]]
                    s[[t_offset]] <- s[[t_offset - m]] + gamma * e[[t]]
                    b[[t]] <- phi * b[[t - 1]] + alpha * beta * e[[t]]
                    l[[t]] <- l[[t - 1]] + b[[t - 1]] + alpha * e[[t]]
                }
                y <- u + e + u_out
            } else {
                for (t in seq(2, ts + 1)) {
                    t_offset <- t + m - 1
                    u[[t]] <- l[[t - 1]] + b[[t - 1]] + s[[t_offset - m]]
                    s[[t_offset]] <- s[[t_offset - m]] + gamma * e[[t]] * u[[t]]
                    b[[t]] <- phi * b[[t - 1]] + alpha * beta * e[[t]] * u[[t]]
                    l[[t]] <- l[[t - 1]] + b[[t - 1]] + alpha * e[[t]] * u[[t]]
                }
                y <- u * unlist(lapply((e + u_out), outlier_multi))
            }
        }
        #Simulating data
        for (t in seq(2, ts + 1)) {
            t_offset <- t + m - 1
            s[[t_offset]] <- gamma * (y[[t]] - l[[t - 1]] - phi * b[[t - 1]]) +
             (1 - gamma) * s[[t_offset - m]]
            l[[t]] <- alpha * (y[[t]] - s[[t_offset - m]]) +
             (1 - alpha) * (l[[t - 1]] + phi * b[[t - 1]])
            b[[t]] <- beta * (l[[t]] - l[[t - 1]]) +
             (1 - beta) * phi * b[[t - 1]]
            y_pred[t + h] <- l[[t]] + phi * b[[t]] + s[t_offset - m + h_m]
        }
    } else if (type %in% c("MNM", "MAM", "MDM")) {
        s[1:4] <- s_m
        if (type %in% c("MNM")) {
            phi <- 0
            for (t in seq(2, ts + 1)) {
                t_offset <- t + m - 1
                u[[t]] <- l[[t - 1]] * s[[t_offset - m]]
                s[[t_offset]] <- s[[t_offset - m]] +
                 (gamma * e[[t]] * u[[t]]) / l[[t - 1]]
                l[[t]] <- l[[t - 1]] +
                 (alpha * e[[t]] * u[[t]]) / s[[t_offset - m]]
            }
            y <- u * unlist(lapply((e + u_out), outlier_multi))
        } else if (type %in% c("MAM")) {
            phi <- 1
            for (t in seq(2, ts + 1)) {
                t_offset <- t + m - 1
                u[[t]] <- ((l[[t - 1]] + b[[t - 1]]) * s[[t_offset - m]])
                s[[t_offset]] <- s[[t_offset - m]] +
                 (gamma * e[[t]] * u[[t]]) / (l[[t - 1]] + b[[t - 1]])
                b[[t]] <- b[[t - 1]] +
                 (alpha * beta * e[[t]] * u[[t]]) / s[[t_offset - m]]
                l[[t]] <- l[[t - 1]] +
                 b[[t - 1]] + (alpha * e[[t]] * u[[t]]) / s[[t_offset - m]]
            }
            y <- u * unlist(lapply((e + u_out), outlier_multi))
        } else {
            for (t in seq(2, ts + 1)) {
                t_offset <- t + m - 1
                u[[t]] <- ((l[[t - 1]] + b[[t - 1]]) * s[[t_offset - m]])
                s[[t_offset]] <- s[[t_offset - m]] +
                 (gamma * e[[t]] * u[[t]]) / (l[[t - 1]] + b[[t - 1]])
                b[[t]] <- phi * b[[t - 1]] +
                 (alpha * beta * e[[t]] * u[[t]]) / s[[t_offset - m]]
                l[[t]] <- l[[t - 1]] + b[[t - 1]] +
                 (alpha * e[[t]] * u[[t]]) / s[[t_offset - m]]
            }
            y <- u * unlist(lapply((e + u_out), outlier_multi))
        }
        for (t in seq(2, ts + 1)) {
            t_offset <- t + m - 1
            s[[t_offset]] <- gamma * (y[[t]] / (l[[t - 1]] +
             phi * b[[t - 1]])) + (1 - gamma) * s[[t_offset - m]]
            l[[t]] <- alpha * (y[[t]] / s[[t_offset - m]]) +
             (1 - alpha) * (l[[t - 1]] + phi * b[[t - 1]])
            b[[t]] <- beta * (l[[t]] - l[[t - 1]]) +
             (1 - beta) * phi * b[[t - 1]]
            y_pred[t + h] <- (l[[t]] + phi * b[[t]]) * s[t_offset - m + h_m]
        }
    } else {
        print("Error: Model is not initialized.")
    }
    return(list("y" = y, "y_pred" = y_pred, "model" = type))
}
get_best_psi <- function(x, model, scale.estimator = "mad", psifun = "Huber", damping = FALSE) {
    l <- c()
    a <- c("Huber", "Bisquare", "Hampel", "Welsh1", "Welsh2")
    for (i in a) {
        m_ahenao <- robets(x,
        model = model,
        scale.estimator = scale.estimator,
        psifun = i,
        damped = damping)
        l[[paste0(i)]] <- m_ahenao$robaicc
    }
    return(which.min(l))
}
simulate_results <- function(i = 1000, m = "ANN", damping = FALSE, out = FALSE) {
    model_type <- m
    iterations <- i
    r <- vector("list", iterations)
    if (is.element(substr(m, 2, 2), c("D"))) {
        substr(model_type, 2, 2) <- "A"
        damping <- TRUE
    }
    for (n in seq(iterations)) {
        m.t <- model(model_type, out = out)  # Outlier flag
        if (model_type %in% c("ADN", "MDN")) {
        time_series <- m.t$y[c(-1, -2)]
        } else {
        time_series <- m.t$y[c(-1)]
        }
        title <- get_best_psi(time_series,
         model = model_type,
         damping = damping)
        mf <- mean(time_series[1:11])
        ml <- mean(time_series[31:length(time_series)])
        mc <- abs(mf - ml) / mf
        r[[n]] <- c(n, names(title), model_type,
        sd(time_series), mf, ml, mc
        )
    }
    data <- do.call(rbind, r)
    colnames(data) <- c("iter", "loss", "model", "std", "mean_first",
     "mean_last", "mean_change")
    data <- transform(data, iter =  as.numeric(iter),
    std = as.numeric(std),
    mean_first = as.numeric(mean_first),
    mean_last = as.numeric(mean_last),
    mean_change = as.numeric(mean_change)
    )
    return(data)
}
# res <- list()
# for (i in seq(10)) {
#     data <- simulate_results(1000, "MNA", out = TRUE)
#     x <- data %>%
#          group_by(loss) %>%
#          summarise(std = median(std),
#           mean_first = median(mean_first),
#           mean_last = median(mean_last),
#           mean_change = median(mean_change),
#           model = n() / 10)
#     res[[i]] <- x
# }
# binded <- bind_rows(res)
# print(binded, n = 100)


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Registered S3 methods overwritten by 'TSA':
  method       from    
  fitted.Arima forecast
  plot.Arima   forecast


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Rcpp

Loading required package: rlang

Loading required package: common



In [2]:
rmse <- function(actual, predicted) {
    sqrt(mean((actual - predicted)^2))
}

In [3]:
prophet_model_forecast <- function(df, h) {
    m <- prophet(df, yearly.seasonality = FALSE,
     daily.seasonality = FALSE, n.changepoints = 24)
    future <- make_future_dataframe(m, periods = h)
    forecast <- predict(m, future)
    yhat <- tail(forecast$yhat, h)
    return (yhat)
}

In [101]:
simulate_prophet_results <- function(m = "ANN", damping = FALSE, out = FALSE) {
    model_type <- m
    h <- 8
    iter <- 100
    if (is.element(substr(m, 2, 2), c("D"))) {
        substr(model_type, 2, 2) <- "A"
        damping <- TRUE
    }
    observed <- c()
    smoothed <- c()
    exp_forecast <- c()
    for (i in seq(iter)){    
        m.t <- model(model_type, out = out)  # Outlier flag
        if (model_type %in% c("ADN", "MDN")) {
            time_series <- m.t$y[c(-1, -2)]
        } else {
            time_series <- m.t$y[c(-1)]
        }
        title <- get_best_psi(
            time_series,
            model = model_type,
            damping = damping
            )
        rob <- robets(
            time_series,
            model = model_type,
            scale.estimator = "mad",
            psifun = names(title),
            damped = damping
            )
        obs.y <- rob$x
        n <- length(obs.y)
        train_obs.y <- head(obs.y, n - h)
        train_pred.y <- head(rob$y_pred, n - h)
        true_obs.y <- tail(obs.y, h)
        true_pred.y <- tail(rob$y_pred, h)
        ts_df <- data.frame(y=train_obs.y, ds = as.Date('2022-01-01') + 0:(n-h-1))
        pred_df <- data.frame(y=train_pred.y, ds = as.Date('2022-01-01') + 0:(n-h-1))
        test_obs.y <- prophet_model_forecast(ts_df, h)
        test_pred.y <- prophet_model_forecast(pred_df, h)
        tryCatch({
            test_exp.y <- c(forecast(rob, h=h)$mean)
        }, error = function(e) {
            test_exp.y <- true_pred.y
        })
        e.obs <- rmse(true_obs.y, test_obs.y)
        e.smoothed <- rmse(true_pred.y, test_pred.y)
        e.exp <- rmse(true_obs.y, test_exp.y)
        observed <- c(observed, e.obs)
        smoothed <- c(smoothed, e.smoothed)
        exp_forecast <- c(exp_forecast, e.exp)

    }
    return (list(obs=observed, sm=smoothed ,exp=exp_forecast))
}

In [102]:
models <- c("AAA", "AAN", "ADA", "ADN", "ANA", "ANN", "MAA", "MAM", "MAN", "MDA", "MDM", "MDN", "MNA", "MNM", "MNN")

In [103]:
simulate_models <- function(mt, out = TRUE) {
    obs.median.errors <- c()
    smoothed.median.errors <- c()
    exp_forecast.median.errors <- c()
    obs.desv <- c()
    smoothed.desv <- c()
    exp_forecast.desv <- c()
    res <- simulate_prophet_results(mt, out=out)
    obs.median.errors <- c(obs.median.errors, median(res$obs))
    obs.desv <- c(obs.desv, sd(res$obs))
    smoothed.desv <- c(smoothed.desv, sd(res$sm))
    smoothed.median.errors <- c(smoothed.median.errors, median(res$sm))
    exp_forecast.median.errors <- c(exp_forecast.median.errors, median(res$exp))
    exp_forecast.desv <- c(exp_forecast.desv, sd(res$exp))
    return (list(obs=obs.median.errors, sm=smoothed.median.errors ,obs.desv=obs.desv,
     sm.desv=smoothed.desv ,exp=exp_forecast.median.errors ,exp.desv=exp_forecast.desv ))
}

In [104]:
outliers.obs <- c()
outliers.sm <- c()
no_outliers.obs <- c()
exp.obs <- c()
exp.no.obs <- c()

outliers.desv <- c()
no_outliers.desv <- c()
outliers.sm.desv <- c()
exp.obs.desv <- c()
exp.no.obs.desv <- c()

for (mt in models){
    o <- simulate_models(mt, out=TRUE)
    no <- simulate_models(mt, out=FALSE)
    outliers.obs <- c(outliers.obs, o$obs)
    no_outliers.obs <- c(no_outliers.obs, no$obs)
    outliers.sm <- c(outliers.sm, o$sm)
    exp.obs <- c(exp.obs, o$exp)
    exp.no.obs <- c(exp.no.obs, no$exp)

    outliers.desv <- c(outliers.desv, o$obs.desv)
    no_outliers.desv <- c(no_outliers.desv, no$obs.desv)
    outliers.sm.desv <- c(outliers.sm.desv, o$sm.desv)
    exp.obs.desv <- c(exp.obs.desv, o$exp.desv)
    exp.no.obs.desv <- c(exp.no.obs.desv, no$exp.desv)
}

In [124]:
results_df <- data.frame (
    models  = models,
    outliers_observed = outliers.obs,
    outliers_observed.desv = outliers.desv,
    outliers_smoothed = outliers.sm,
    outliers_smoothed.desv = outliers.sm.desv,
    no_outliers = no_outliers.obs,
    no_outliers.desv = no_outliers.desv,
    exp_out = exp.obs,
    exp_out.desv = exp.obs.desv,
    exp_no_out = exp.no.obs,
    exp.no.obs.desv = exp.no.obs.desv
    )
results_df

models,outliers_observed,outliers_observed.desv,outliers_smoothed,outliers_smoothed.desv,no_outliers,no_outliers.desv,exp_out,exp_out.desv,exp_no_out,exp.no.obs.desv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAA,0.11917146,0.1917066,0.029550386,0.011771178,0.023727924,0.001262036,0.405017037,0.10211864,0.397362689,0.0103561782
AAN,0.14264254,0.1872081,0.004863264,0.003593141,0.003899601,0.002164456,0.402139231,0.13156623,0.400238906,0.0090553622
ADA,0.12930792,0.1514640,0.029384647,0.021153601,0.023803680,0.001423182,0.375909829,0.07602456,0.377047308,0.0097648566
ADN,0.12140549,0.1577608,0.005905255,0.004465344,0.003797698,0.002092029,0.390263447,0.07897604,0.385351574,0.0125079200
ANA,0.13310392,0.1965410,0.027337494,0.011074938,0.023217786,0.001174027,0.026053755,0.20130660,0.026082607,0.0029817910
ANN,0.10926754,0.1858476,0.004336995,0.002210851,0.003666417,0.001820980,0.002586334,0.18978567,0.002295537,0.0007030536
MAA,0.20381561,0.4299836,0.038176369,0.021936449,0.025301569,0.004327105,0.409429575,0.30870425,0.398133062,0.0205029591
MAM,0.14696733,0.4796857,0.075614328,0.026875457,0.069309824,0.004233304,0.413424772,0.36484288,0.393693218,0.0214386011
MAN,0.19769859,0.4626117,0.011386914,0.005132511,0.009521392,0.004482341,0.406666897,0.33320120,0.400033227,0.0187550438


In [ ]:
results_df$outliers_observed <- fapply(results_df$outliers_observed, "%.3f")
results_df$outliers_smoothed <- fapply(results_df$outliers_smoothed, "%.3f")
results_df$no_outliers <- fapply(results_df$no_outliers, "%.3f")
results_df

ERROR: Error in sprintf(fmt, x): invalid format '%.4f'; use format %s for character objects
